# Week 11: Grid Search Model Example

In [1]:
# use the penguins dataset; species = y; explore the dataset
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder

df = sns.load_dataset('penguins')
print(df.info())
df.dropna(how='any', inplace=True)
X_train, X_test, y_train, y_test = train_test_split(df.drop('species', axis=1), df['species'], test_size=0.25, random_state=42)

X_train['sex'] = X_train['sex'].map({'Female':1,'Male':0})
X_test['sex'] = X_test['sex'].map({'Female':1,'Male':0})

ohe = OneHotEncoder(categories='auto', drop='first', sparse=False)

ohe_train = ohe.fit_transform(X_train[['island']])
ohe_train = pd.DataFrame(ohe_train, columns=ohe.get_feature_names_out(['island']))
ohe_train.index = X_train.index
X_train = X_train.join(ohe_train)
X_train.drop(['island'], axis=1, inplace=True)

ohe_test = ohe.transform(X_test[['island']])
ohe_test = pd.DataFrame(ohe_test, columns=ohe.get_feature_names_out(['island']))
ohe_test.index = X_test.index
X_test = X_test.join(ohe_test)
X_test.drop(['island'], axis=1, inplace=True)

y_train.value_counts()
y_train = y_train.map({'Adelie':0,'Gentoo':1, 'Chinstrap':2})
y_test = y_test.map({'Adelie':0,'Gentoo':1, 'Chinstrap':2})

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 344 entries, 0 to 343
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            344 non-null    object 
 1   island             344 non-null    object 
 2   bill_length_mm     342 non-null    float64
 3   bill_depth_mm      342 non-null    float64
 4   flipper_length_mm  342 non-null    float64
 5   body_mass_g        342 non-null    float64
 6   sex                333 non-null    object 
dtypes: float64(4), object(3)
memory usage: 18.9+ KB
None


In [2]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

hyperparameters = {
            'n_estimators': [50, 200],
            'criterion': ['entropy', 'gini'],
            'max_depth': [3, 4],
            'max_leaf_nodes': [7, 9],
            'bootstrap': [True, False]
            }

grid_search = GridSearchCV(estimator = RandomForestClassifier(),
                           param_grid = hyperparameters,
                           scoring = 'accuracy',
                           cv = 10)

grid_search = grid_search.fit(X_train, y_train)

best_accuracy = grid_search.best_score_
best_parameters = grid_search.best_params_

print('best accuracy', best_accuracy)
print('best parameters', best_parameters)

best accuracy 0.992
best parameters {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 4, 'max_leaf_nodes': 9, 'n_estimators': 200}


## Our Final Model with Best (Hyper)Parameters

best_params from gridsearch: {'bootstrap': True, 'criterion': 'entropy', 'max_depth': 3, 'max_leaf_nodes': 7, 'n_estimators': 50}

<pre>
model = RandomForestClassifier(
            n_estimators=50,
            criterion='entropy,
            max_depth=3,
            max_leaf_nodes=7,
            bootstrap=True
            )
</pre> 

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(n_estimators = 50, 
                               criterion = 'entropy', 
                               max_depth = 3, 
                               max_leaf_nodes = 7, 
                               bootstrap = True, 
                               random_state = 42)

model.fit(X_train, y_train)
predictions = model.predict(X_test)
print(accuracy_score(y_test, predictions))

0.9880952380952381


In [4]:
# build your final RandomForestClassifier model here using set_params and best_params; provide an accuracy score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model = RandomForestClassifier(random_state = 42).set_params(**best_parameters) # * args, ** kwargs
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print(accuracy_score(y_test, predictions))
print(model.get_params)

0.9880952380952381
<bound method BaseEstimator.get_params of RandomForestClassifier(criterion='entropy', max_depth=4, max_leaf_nodes=9,
                       n_estimators=200, random_state=42)>
